In [7]:
!git clone https://github.com/Srija616/DeepLearning_Assignment_2.git

Cloning into 'DeepLearning_Assignment_2'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 52 (delta 21), reused 28 (delta 8), pack-reused 0
Receiving objects: 100% (52/52), 9.76 KiB | 208.00 KiB/s, done.
Resolving deltas: 100% (21/21), done.


In [8]:
!nvidia-smi

Sat Apr  6 18:10:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.07             Driver Version: 535.161.07   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090 Ti     Off | 00000000:08:00.0  On |                  Off |
|  0%   46C    P8              15W / 450W |    736MiB / 24564MiB |      6%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install wandb
import wandb

In [2]:
!wandb login --relogin ded42ee90252a06479e4661a4a881b7b55422258

wandb: Appending key for api.wandb.ai to your netrc file: /home/vg/.netrc


In [ ]:
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip

In [ ]:
!unzip "nature_12K.zip"

In [ ]:
!pip install lightning torchmetrics

In [3]:
import os
import wandb
import random
import numpy as np

In [4]:
import torch
from torch import nn,optim
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset, random_split, Subset
import torchvision
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torchmetrics import Accuracy
torch.manual_seed(42)

In [5]:
path = os.path.dirname(os.path.realpath('__file__'))
train_path = os.path.join(path, 'inaturalist_12K/train/')
test_path = os.path.join(path, 'inaturalist_12K/val/')
print (train_path, test_path)

/home/vg/ttsteam/repos/srija/inaturalist_12K/train/ /home/vg/ttsteam/repos/srija/inaturalist_12K/val/


In [6]:
def preprocess_data(train_path, test_path, data_aug=False):
    if data_aug == True:
        transforms = T.Compose([T.RandomResizedCrop((224, 224)),
                            T.RandomHorizontalFlip(p=0.5),
                            T.RandomVerticalFlip(p=0.5),
                            T.RandomRotation(degrees=45),
                            T.ToTensor(),
                            T.Normalize(mean=[0, 0, 0], std=[1, 1, 1])
                ])
    else:
        transforms = T.Compose([T.RandomResizedCrop((224, 224)),
                    T.ToTensor(),
                    T.Normalize(mean=[0, 0, 0], std=[1, 1, 1])])

    dataset = ImageFolder(train_path, transform=transforms)
    test_dataset = ImageFolder(test_path, transform=transforms)

    train_size = int(0.8*len(dataset))
    val_size = (len(dataset) - train_size)

    class_labels = [label for _, label in dataset]
    target = np.array(class_labels)

    from sklearn.model_selection import StratifiedShuffleSplit
    stratified_splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

    for train_idx, val_idx in stratified_splitter.split(np.zeros(len(target)), target):
        train_dataset = Subset(dataset, train_idx)
        val_dataset = Subset(dataset, val_idx)

    return train_dataset, val_dataset, test_dataset

In [5]:
main_config = {
    "num_filters": 32,
    "filter_size": 5,
    "neurons_dense": 32,
    "activation": 'relu',
    "batch_norm": False,
    "filter_org": 0.5,
    "data_aug": True,
    "dropout": 0.1,
    "batch_size": 16,
    }

In [6]:
# train_dataset, val_dataset, test_dataset = preprocess_data(train_path, test_path, main_config["data_aug"])

In [7]:
import sys
sys.path.append("/home/vg/ttsteam/repos/srija/DeepLearning_Assignment_2/models")
from model import Model
import pytorch_lightning as pl

in_channels = 3
image_shape = (1,3,224,224)
num_classes = 10
patience_epochs = 7



# model = Model(in_channels=in_channels,
#               num_filters= config['num_filters'],
#               filter_size = config['filter_size'],
#               activation = config['activation'],
#               neurons_dense = config['neurons_dense'],
#               image_shape = image_shape,
#               batch_norm = config['batch_norm'],
#               filter_org = config["filter_org"],
#               classes=num_classes,
#               dropout = config["dropout"])

In [13]:
from lightning.pytorch.loggers import WandbLogger
def train_cnn():
    wandb.init(project='DL-Assignment2')
    wandb_logger = WandbLogger(project="DL-Assignment2")
    config = wandb.config
    wandb.run.name = f"num_filters-{config.num_filters}--activation-{config.activation}--lr-{config.lr}--neurons_dense-{config.neurons_dense}--batch_norm-{config.batch_norm}--filter_org-{config.filter_org}--dropout-{config.dropout}--model"
    train_dataset, val_dataset, test_dataset = preprocess_data(train_path, test_path, config["data_aug"])
    print ('Preprocessing data completed')
    
    model = Model(in_channels=in_channels,
            num_filters= config['num_filters'],
            filter_size = config['filter_size'],
            activation = config['activation'],
            neurons_dense = config['neurons_dense'],
            image_shape = image_shape,
            batch_norm = config['batch_norm'],
            filter_org = config["filter_org"],
            classes=num_classes,
            dropout = config["dropout"],
            batch_size = config["batch_size"],
            lr = config['lr'],
            train_dataset = train_dataset,
            test_dataset=test_dataset,
            val_dataset=val_dataset)

    

    early_stop_callback = pl.callbacks.EarlyStopping(monitor='val_loss', patience=patience_epochs, mode='min')
    trainer = pl.Trainer(devices=-1, max_epochs=40, precision='bf16-mixed', logger=wandb_logger, callbacks=[early_stop_callback])
    print('Trainer initialized')

    trainer.fit(model)


In [14]:
# wandb.login()

sweep_config = {
    "name" : "DL-Assignment2",
    "method" : "bayes",
    'metric': {
        'name': 'val_acc',
        'goal': 'maximize'
    },
    "parameters" : {
        "num_filters":{
            "values":[32,64,128]
        },
        "lr":{
            "values":[0.001,0.0001,0.0003,0.0005]
        },
        "filter_size": {
            "values":[3,5]
        },
        "neurons_dense":{
            "values":[32, 64]
        },
        'activation': {
            'values': ['relu', 'gelu', 'silu', 'mish']
        },
        "batch_norm": {
              "values": [True,False]
        },
        'filter_org': {
            'values': [1, 2, 0.5]
        },
        'learning_rate':{
            "values": [0.001,0.0001,0.0003,0.0005]
        },
        "dropout": {
            "values": [0,0.1,0.2,0.3]
        },
        "data_aug": {
              "values": [True,False]
        },
        "batch_size": {
            "values": [16, 32, 64]
        },
    }
}

In [15]:
sweep_id = wandb.sweep(sweep_config, project="DL-Assignment2", entity="srija17199")
print('sweep_id: ', sweep_id)
wandb.agent(sweep_id, function=train_cnn)

Create sweep with ID: sukf499h
Sweep URL: https://wandb.ai/srija17199/DL-Assignment2/sweeps/sukf499h
sweep_id:  sukf499h


wandb: Agent Starting Run: 30i58ew3 with config:
wandb: 	activation: relu
wandb: 	batch_norm: True
wandb: 	batch_size: 16
wandb: 	data_aug: True
wandb: 	dropout: 0.2
wandb: 	filter_org: 0.5
wandb: 	filter_size: 3
wandb: 	learning_rate: 0.0001
wandb: 	lr: 0.001
wandb: 	neurons_dense: 64
wandb: 	num_filters: 128
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/vg/.miniconda3/envs/dl_ass2/lib/python3.9/site-packages/lightning/pytorch/loggers/wandb.py:391: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type               | Params
---------------------------------------------------
0 

Preprocessing data completed
Trainer initialized


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]